# EFT Chatbot

In [11]:
from typing import Dict, Optional
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableSequence
from langchain_core.output_parsers import StrOutputParser
from langchain_core.memory import BaseMemory
from langchain.memory import ConversationBufferMemory
import google.generativeai as genai
import os
from dotenv import load_dotenv
import time
from typing import Any, List, Optional, Dict
import logging
from langchain.chains import LLMChain
from typing import Dict, Any, Optional, Callable
from langchain.llms.base import LLM
from langchain.memory import ConversationBufferWindowMemory  # Import for reinitializing memory


logger = logging.getLogger(__name__)

# Custom Gemini LLM using Google GenAI client

In [8]:
class GeminiLLM(LLM):
    """
    Custom LLM that uses the Gemini API via the Google GenAI client.
    GEMINI_API_KEY is loaded from the .env file.
    """
    def __init__(self):
        super().__init__()
        # Configure the Gemini API
        genai.configure(api_key=os.getenv("GEMINI_API_KEY"))
        
    @property
    def _llm_type(self) -> str:
        return "gemini"

    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        model = genai.GenerativeModel('gemini-1.5-pro')
        
        # Handle general queries differently
        if self._is_general_query(prompt):
            return self._handle_general_query(prompt)
            
        response = model.generate_content(prompt)
        return response.text

    def _is_general_query(self, prompt: str) -> bool:
        """Check if the prompt is a general greeting or simple question."""
        general_queries = [
            "hello", "hi", "hey", "greetings", "how are you", 
            "how's it going", "what's up", "good morning", 
            "good afternoon", "good evening", "help", "what is eft"
        ]
        
        prompt_lower = prompt.lower().strip()
        
        for query in general_queries:
            if prompt_lower.startswith(query) or prompt_lower == query:
                return True
        return False
        
    def _handle_general_query(self, prompt: str) -> str:
        """Provide a concise, friendly response to general queries."""
        prompt_lower = prompt.lower().strip()
        
        if any(greeting in prompt_lower for greeting in ["hello", "hi", "hey", "greetings"]):
            return "Hello! I'm your EFT Therapy Assistant. How are you feeling today? I'm here to help guide you through emotional freedom techniques."
        elif "how are you" in prompt_lower or "how's it going" in prompt_lower or "what's up" in prompt_lower:
            return "I'm here and ready to support you with EFT techniques. How are you feeling today? Is there something specific you'd like to work on?"
        elif any(greeting in prompt_lower for greeting in ["good morning", "good afternoon", "good evening"]):
            return "Thank you, and the same to you! I'm your EFT Therapy Assistant. How can I support your emotional wellbeing today?"
        elif "help" in prompt_lower:
            return ("I'm here to help you with Emotional Freedom Techniques (EFT). "
                    "You can share how you're feeling, describe emotional challenges, or ask for guidance on specific issues. "
                    "I'll provide personalized tapping sequences to help you process emotions and find relief.")
        elif "what is eft" in prompt_lower:
            return ("EFT (Emotional Freedom Techniques) is a powerful self-help method that combines elements of cognitive therapy with acupressure. "
                    "By tapping on specific meridian points while focusing on an issue, EFT helps release emotional blockages and reduce stress. "
                    "It's often called 'psychological acupressure' and can help with anxiety, stress, trauma, and other emotional challenges.")
        else:
            return "I'm your EFT Therapy Assistant. How can I support you today?"

    @property
    def _identifying_params(self) -> Dict[str, Any]:
        return {
            "model": "gemini-pro",
            "temperature": 0.7,
        }


# Define the EFT Therapy System using LangChain

In [9]:

class EFTTherapySystem:
    def __init__(self):
        logger.debug("Initializing EFTTherapySystem.")
        self.llm = GeminiLLM()
        self.memory = ConversationBufferWindowMemory(
            memory_key="chat_history",
            return_messages=True,
            k=10  # Keep last 10 messages
        )
        
        self.decomposition_prompt = PromptTemplate(
            input_variables=["input", "chat_history"],
            template="""You are a Task Decomposition Specialist for EFT (Emotional Freedom Techniques) therapy.
            
Your role is to break down complex EFT queries into clear, actionable subtasks.

Given the following query and conversation history, break down the query into specific therapeutic needs:

Query: {input}

Conversation History: {chat_history}

List each subtask on a new line, focusing on key therapeutic needs. Be sensitive to emotional cues and underlying issues:
"""
        )
        self.decomposition_chain = LLMChain(llm=self.llm, prompt=self.decomposition_prompt)
        
        self.expert_prompt = PromptTemplate(
            input_variables=["input", "subtasks", "chat_history"],
            template="""You are an EFT Therapy Expert with years of experience in Emotional Freedom Techniques.
            
You help patients address emotional challenges through tapping techniques, combining clinical expertise with warm empathy.

Based on the following subtasks identified for this query, provide expert EFT guidance addressing each component:

Original Query: {input}

Subtasks:
{subtasks}

Conversation History: {chat_history}

For each subtask, provide:
1. A brief validation of the feeling or concern
2. Clear tapping instructions with specific phrases to say during tapping
3. A comforting rationale for why this approach helps

Use a warm, conversational tone as if you're speaking directly to the person. Include gentle encouragement and normalize their experience:
"""
        )
        self.expert_chain = LLMChain(llm=self.llm, prompt=self.expert_prompt)
        
        self.coordinator_prompt = PromptTemplate(
            input_variables=["input", "expert_guidance", "chat_history"],
            template="""You are an EFT Session Coordinator skilled at creating conversational, empathetic therapeutic responses.
            
Your task is to create a cohesive, empathetic response that feels like a real therapist is talking directly to the client.

Original Query: {input}

Expert Guidance:
{expert_guidance}

Conversation History: {chat_history}

Create a response that:
1. Opens with a warm, personalized greeting that acknowledges their feelings
2. Presents the EFT guidance in a conversational, easy-to-follow format
3. Uses supportive language and occasional questions to engage them
4. Closes with encouragement and an invitation to share how they feel after trying the techniques

The response should feel like a caring conversation, not a clinical instruction manual. Use natural language with some variation in sentence structure, occasional gentle questions, and empathetic observations:
"""
        )
        self.coordinator_chain = LLMChain(llm=self.llm, prompt=self.coordinator_prompt)
    
    def process_query(self, user_query: str, progress_callback: Optional[Callable] = None) -> Dict[str, str]:
        memory_vars = self.memory.load_memory_variables({})
        chat_history = memory_vars.get("chat_history", "")
        
        if self.llm._is_general_query(user_query):
            response = self.llm._handle_general_query(user_query)
            self.memory.save_context({"input": user_query}, {"output": response})
            return {
                "subtasks": "General query",
                "expert_guidance": "Direct response",
                "final_response": response
            }
        
        if progress_callback:
            progress_callback("Listening to your concerns...", 0.2)
        
        decomposition_result = self.decomposition_chain.run({
            "input": user_query,
            "chat_history": chat_history
        })
        
        if progress_callback:
            progress_callback("Preparing personalized EFT guidance...", 0.5)
        
        expert_result = self.expert_chain.run({
            "input": user_query,
            "subtasks": decomposition_result,
            "chat_history": chat_history
        })
        
        if progress_callback:
            progress_callback("Crafting a supportive response for you...", 0.8)
        
        final_result = self.coordinator_chain.run({
            "input": user_query,
            "expert_guidance": expert_result,
            "chat_history": chat_history
        })
        
        if progress_callback:
            progress_callback("Ready with your guidance", 1.0)
        
        self.memory.save_context({"input": user_query}, {"output": final_result})
        
        return {
            "subtasks": decomposition_result,
            "expert_guidance": expert_result,
            "final_response": final_result
        }


# Main Chatbot Class with Memory


In [ ]:
def main():
    eft_system = EFTTherapySystem()

    print("Welcome to the EFT Therapy System. Type 'exit' to quit.\n")
    while True:
        user_input = input("User: ")
        if user_input.lower() in ["exit", "quit"]:
            break
        try:
            answer = eft_system.process_query(user_input)
            print("Assistant:\n", answer, "\n")
        except Exception as e:
            print("Error:", e)
            import traceback

            traceback.print_exc()


if __name__ == "__main__":
    main()

C:\Users\Admin\AppData\Local\Temp\ipykernel_25768\3661093643.py:5: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  self.memory = ConversationBufferWindowMemory(
C:\Users\Admin\AppData\Local\Temp\ipykernel_25768\3661093643.py:26: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  self.decomposition_chain = LLMChain(llm=self.llm, prompt=self.decomposition_prompt)


Welcome to the EFT Therapy System. Type 'exit' to quit.

Assistant:
 {'subtasks': 'General query', 'expert_guidance': 'Direct response', 'final_response': "Hello! I'm your EFT Therapy Assistant. How are you feeling today? I'm here to help guide you through emotional freedom techniques."} 



C:\Users\Admin\AppData\Local\Temp\ipykernel_25768\3661093643.py:93: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  decomposition_result = self.decomposition_chain.run({
